In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import datetime

def get_time_str():
    now = datetime.datetime.now()
    tag = [now.year, now.month, now.day, now.hour, now.minute, now.second]
    return "_".join(map(str, tag))

'2019_6_14_5_27_9'

## Connect to DB

In [2]:
def connect_to_postgresql(user, password):
    connect_string = "dbname='sofi_dw' user='{0}' host='dw-db.sofi.com' password='{1}'".format(user, password)
    try:
        conn = psycopg2.connect(connect_string)
        return conn
    except:
        print ("Unable to connect to the database")
        
def connect_to_postgresql_sdm(port):
    connect_string = "dbname='sofi_dw' host='localhost' port = {0}".format(port)
    try:
        conn = psycopg2.connect(connect_string)
        return conn
    except:
        print ("Unable to connect to the database")
        
def close_connection_to_postgresql(conn):
    conn.close()
    
def postgresql_to_list(query, conn):
    cur = conn.cursor()
    cur.execute(query)
    data = cur.fetchall()
    close_connection_to_postgresql(conn)
    return data

def postgresql_to_df(query, conn):
    dataframe = psql.read_sql(query, conn)
    close_connection_to_postgresql(conn)
    return dataframe

def postgresql_sql_executor(query, conn):
    cur = conn.cursor()
    cur.execute(query)
    conn.commit()
    
def connect_to_postgresql_sdm_reporting(port):
    connect_string = "dbname='sofi' host='127.0.0.1' port = {0}".format(port)
    try:
        conn = psycopg2.connect(connect_string)
        return conn
    except:
        print ("Unable to connect to the database")

In [3]:
conn = connect_to_postgresql_sdm(15501)

query = "select * \
from banking_transactions_facts \
left join banking_transactions_details d2 on banking_transactions_facts.banking_transaction_details_id = d2.banking_transaction_details_id \
limit 500;"

%time bank_trans = postgresql_to_df(query, conn)

CPU times: user 69.8 ms, sys: 1.61 ms, total: 71.4 ms
Wall time: 629 ms


## Upload to S3

In [1]:
# upload
import boto3

BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/' # replace with your object key
Filename = 'startup.sh'
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).upload_file(Filename, KEY + 'startup.sh')

In [2]:
# upload
import boto3

BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/money-risk-models/customer-risk-model/models/' # replace with your object key
Filename = 'money-risk-models/money-customer-risk/models/customer_risk_target_thomas.pkl'
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).upload_file(Filename, KEY + 'customer_risk_target_thomas.pkl')

## Connect to S3

In [ ]:
# download
import boto3

BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/data/measurement_filtered_parquet' # replace with your object key

s3 = boto3.resource('s3')

s3.Bucket(BUCKET_NAME).download_file(KEY, 'measurement_filtered_parquet')


In [2]:
'https://sofi-data-science.s3-us-west-2.amazonaws.com/jxu/instance_backups/backup_7_9_code.zip'

# download
import boto3

BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/instance_backups/backup_7_5.zip' # replace with your object key

s3 = boto3.resource('s3')

s3.Bucket(BUCKET_NAME).download_file(KEY, 'code_backup')


In [ ]:
# read from s3
modeling_df = pd.read_feather('s3://sofi-data-science/jxu/data/acc-level-modeling-df.feather')

In [5]:
# upload
import boto3

BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/data_dump/2020-08-29-customer-data' # replace with your object key
Filename = 'RNN_loader.ipynb'
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).upload_file(Filename, KEY + 'RNN_loader.ipynb')

In [11]:
# upload
import boto3

file = 'modeling_df_trans_w_tcn_pred.feather'
BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/TCN_CLF_TRANS/lgbm_validation/v2/{}'.format(file) # replace with your object key
LOCAL = 'data/TCN_CLF_customer_risk_score/v2/modeling_df_trans_w_tcn_pred.feather'
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).upload_file(LOCAL, KEY)

## Backup the Cur Dir to S3

In [ ]:
!zip -r backup.zip *

  adding: data/ (stored 0%)
  adding: data/df_measurement_2019_1_1.feather (deflated 80%)
  adding: data/measurement_filtered_parquet (deflated 0%)
  adding: data/acc-level-modeling-meas-features.feather (deflated 78%)
  adding: data/TCN_CLF_deposit_risk_model/ (stored 0%)
  adding: data/TCN_CLF_deposit_risk_model/.dill (deflated 86%)
  adding: data/TCN_CLF_deposit_risk_model/TCN_CLF_TRANS_DATA_inbal_v2.dill (deflated 76%)
  adding: data/TCN_CLF_deposit_risk_model/TCN_CLF_TRANS_DATA_inbal_v3.dill (deflated 72%)
  adding: data/TCN_CLF_deposit_risk_model/features_dict_v1.dill (deflated 42%)
  adding: data/TCN_CLF_deposit_risk_model/df_test_v2.dill (deflated 81%)
  adding: data/TCN_CLF_deposit_risk_model/df_train_v1.dill (deflated 82%)
  adding: data/TCN_CLF_deposit_risk_model/df_test_v1.dill (deflated 82%)
  adding: data/TCN_CLF_deposit_risk_model/df_train_v2.dill (deflated 81%)
  adding: data/TCN_CLF_deposit_risk_model/df_test_v3.dill (deflated 83%)
  adding: data/TCN_CLF_deposit_risk_m

In [13]:
# make a zip for all the code and notebooks
!zip -r backup.zip *.ipynb SM* 

	zip warning: name not matched: *.ipynb
	zip warning: name not matched: SM*

zip error: Nothing to do! (try: zip -r backup.zip . -i *.ipynb SM*)


In [16]:
os.getcwd()

'/home/ec2-user/SageMaker/projects-framework'

In [1]:
!tar -czf rdsutils.tar.gz '/home/ec2-user/SageMaker/projects-framework/rdsutils/examples'


tar --exclude='./data'  -czf deposit_risk_model_v2_ach.tar '/home/ec2-user/SageMaker/projects-model/deposit-risk-model-v2-ach'
tar --exclude='./to_script'  -czf deposit_risk_model_v2_ach.tar '/home/ec2-user/SageMaker/projects-model/deposit-risk-model-v2-ach/notebooks'


tar: Removing leading `/' from member names
^C


In [2]:
!zip -r projects-framework.zip *.ipynb '/home/ec2-user/SageMaker/projects-framework/rdsutils/examples'

  adding: load-save-s3.ipynb (deflated 81%)
  adding: planning.ipynb (deflated 57%)
  adding: Toolbox.ipynb (deflated 92%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/ (stored 0%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/woe.ipynb (deflated 82%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/score-alignment.ipynb (deflated 37%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/feature-selection.ipynb (deflated 60%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/scratch-work.ipynb (deflated 79%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/swap-set-analysis.ipynb (deflated 33%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/artifact/ (stored 0%)
  adding: home/ec2-user/SageMaker/projects-framework/rdsutils/examples/artifact/gains_table_example.csv (deflated 56%)
  adding: home/ec2-user/SageMaker/projects-framework

In [ ]:
# upload
import boto3

BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/instance_backups/' # replace with your object key
Filename = 'backup.zip'
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).upload_file(Filename, KEY + 'backup_7_24_all.zip')

In [2]:
import os

os.listdir('data/TCN_CLF_customer_risk_score/inferred_on_full/')

['df_train_acct.feather',
 'tcn.pth',
 'modeling_df.feather',
 'readme.txt',
 'df_test_acct.feather',
 '.ipynb_checkpoints']

In [6]:
# upload

file = 'transactions_df.feather'
BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/TCN_CLF_TRANS/trained_on_full_transactions_data/{}'.format(file) # replace with your object key
Filename = 'data/TCN_CLF_customer_risk_score/inferred_on_full/{}'.format(file)
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).upload_file(Filename, KEY)        

In [5]:
# upload
import os
import boto3

for file in os.listdir('data/TCN_CLF_customer_risk_score/inferred_on_full/'):
    if file != '.ipynb_checkpoints':
        print(file)
        BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
        KEY = 'jxu/TCN_CLF_TRANS/trained_on_full_transactions_data/{}'.format(file) # replace with your object key
        Filename = 'data/TCN_CLF_customer_risk_score/inferred_on_full/{}'.format(file)
        s3 = boto3.resource('s3')
        s3.Bucket(BUCKET_NAME).upload_file(Filename, KEY)        

df_train_acct.feather
tcn.pth
modeling_df.feather
readme.txt
df_test_acct.feather


In [14]:
# upload
import boto3

file = 'tcn.dill'
BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
KEY = 'jxu/TCN_CLF_TRANS/lgbm_validation/v2/{}'.format(file) # replace with your object key
Filename = 'model/TCN_CLF_TRANS_basic_8/{}'.format(file)
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).upload_file(Filename, KEY)

In [18]:
# download
import boto3

filenames = [#'modeling_df_trans_with_tcn_pred.feather',
#              'tcn.pth',
#              'modeling_df_acct.feather',
#              'df.feather',
#              'modeling_df_acct_with_full_validation_preds.feather',
#              'df_test_acct.feather',
#              'transactions_df.feather',
             'v2_TCN_customer_risk_model_validation.ipynb',]
#              'modeling_df_trans.feather',
#              'df_test_trans.feather',
#              'modeling_df_trans_with_full_validation_preds.feather']#,
#              'df_train_trans.feather',
#              'df_train_acct.feather']
    
for file in filenames:
    BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
    KEY = 'jxu/TCN_CLF_TRANS/lgbm_validation/v2/{}'.format(file) # replace with your object key
    LOCAL = 'data/TCN_CLF_customer_risk_score/v2/{}'.format(file)
    s3 = boto3.resource('s3')
    s3.Bucket(BUCKET_NAME).download_file(KEY, LOCAL)

In [20]:
# download
import boto3

filenames = ['v2_TCN_customer_risk_model_validation.ipynb',]
    
for file in filenames:
    BUCKET_NAME = 'sofi-data-science' # replace with your bucket name
    KEY = 'jxu/TCN_CLF_TRANS/lgbm_validation/v2/{}'.format(file) # replace with your object key
    LOCAL = 'SM_TCN_CLF_TRANS/{}'.format(file)
    s3 = boto3.resource('s3')
    s3.Bucket(BUCKET_NAME).download_file(KEY, LOCAL)

## Seaborn Plotting config

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = sns.countplot(data_gt5_len)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90);
ax.set_xlabel('num measurements')
ax.set_title('freq of num measurements')

## Check Number of Cores

In [1]:
import threading
threading.activeCount()

5

## Get Size of

In [1]:
import sys
import psutil
# utility functions
def memory_usage():
    print('total memory: {} GB'.format(psutil.virtual_memory()[0]/1000000000))
    print('available memory: {} GB'.format(psutil.virtual_memory()[1]/1000000000))
    print('used memory: {} GB'.format(psutil.virtual_memory()[3]/1000000000))
    print('free memory: {} GB'.format(psutil.virtual_memory()[4]/1000000000))
    print('active memory: {} GB'.format(psutil.virtual_memory()[5]/1000000000))
    print('inactive memory: {} GB'.format(psutil.virtual_memory()[6]/1000000000))
    print('buffers memory: {} GB'.format(psutil.virtual_memory()[7]/1000000000))
    print('cached memory: {} GB'.format(psutil.virtual_memory()[8]/1000000000))
    
def size_in_memory(df):
    print(df.shape)
    print('size in memory: {} MB'.format(sys.getsizeof(df)/1000000))
    return df

In [1]:
import pandas as pd

In [3]:
memory_usage()

total memory: 401 GB
available memory: 397 GB
used memory: 1 GB
free memory: 397 GB
active memory: 1 GB
inactive memory: 1 GB
buffers memory: 0 GB
cached memory: 1 GB
